In [6]:
"""
Practical pipelines to improve 16-class (4-qubit) readout classification
from fixed IQ points (one complex number per qubit per shot).

Implements 3 practical strategies:
  A) Baseline multiclass SVM (RBF or custom ZZ kernel)
  B) Hierarchical decoding:
       - 4 binary classifiers for q1..q4 (multi-output)
       - combine to 16-class via bit packing
       - optional meta-corrector to fix correlated errors using predicted probabilities
  C) MLP discriminator (multiclass)

Feature engineering options:
  - encoding="cartesian": [Re1, Im1, ..., Re4, Im4]  -> 8 dims
  - encoding="polar":     [amp1, sin(phi1), cos(phi1), ..., amp4, sin(phi4), cos(phi4)] -> 12 dims
  - encoding="polar_cross": polar + cross terms Re(z_i z_j*) and Im(z_i z_j*) for i<j -> 12 + 12 = 24 dims

Scaling options:
  - scaler="robust" | "standard" | "none"

Data files:
  - dataset_X.txt (N,4) complex
  - dataset_y.txt (N,4) bits [q1 q2 q3 q4]
"""
import numpy as np
from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

# ---------------------------
# Loading
# ---------------------------
def load_dataset(X_path="dataset_X.txt", y_path="dataset_y.txt"):
    X = np.loadtxt(X_path, dtype=complex)   # (N,4)
    y = np.loadtxt(y_path, dtype=int)       # (N,4)
    return X, y

def bits_to_int_labels(y_bits):
    """
    Convert [q1,q2,q3,q4] -> integer 0..15, q1 LSB, q4 MSB.
    """
    q1, q2, q3, q4 = (y_bits[:, k] for k in range(4))
    return (q1 + 2*q2 + 4*q3 + 8*q4).astype(int)

# ---------------------------
# Feature engineering
# ---------------------------
def features_cartesian(Xc):
    # (N,4) complex -> (N,8)
    return np.hstack([Xc.real, Xc.imag])

def features_polar(Xc):
    # (N,4) complex -> (N,12): [amp, sin(phi), cos(phi)] per qubit
    amp = np.abs(Xc)
    phi = np.angle(Xc)
    return np.hstack([amp, np.sin(phi), np.cos(phi)])

def features_cross_terms(Xc):
    """
    Cross terms from z_i * conj(z_j): capture relative phase & correlation
    For each pair (i<j): produce Re and Im -> 2 * C(4,2) = 12 dims
    """
    N = Xc.shape[0]
    feats = []
    for i, j in combinations(range(4), 2):
        prod = Xc[:, i] * np.conjugate(Xc[:, j])
        feats.append(prod.real.reshape(N, 1))
        feats.append(prod.imag.reshape(N, 1))
    return np.hstack(feats)  # (N,12)

def make_features(Xc, encoding="cartesian"):
    if encoding == "cartesian":
        return features_cartesian(Xc)
    if encoding == "polar":
        return features_polar(Xc)
    if encoding == "polar_cross":
        return np.hstack([features_polar(Xc), features_cross_terms(Xc)])
    raise ValueError("encoding must be 'cartesian'|'polar'|'polar_cross'")

def make_scaler(scaler="robust"):
    if scaler == "robust":
        return RobustScaler()
    if scaler == "standard":
        return StandardScaler()
    if scaler == "none":
        return None
    raise ValueError("scaler must be 'robust'|'standard'|'none'")

# ---------------------------
# ZZ kernel (optional)
# ---------------------------
from scipy.linalg import pinv

def make_zz_kernel(gamma_single=0.05, gamma_pair=0.05):
    """
    ZZ-inspired kernel for real feature vectors:
        K = Π_k cos(γs (x_k - y_k))  ×  Π_{i<j} cos(γp (x_i x_j - y_i y_j))
    """
    def zz_kernel(X, Y=None):
        X = np.asarray(X, float)
        if Y is None:
            Y = X
        else:
            Y = np.asarray(Y, float)

        diff = X[:, None, :] - Y[None, :, :]
        K_single = np.cos(gamma_single * diff).prod(axis=2)

        K_pair = np.ones_like(K_single)
        d = X.shape[1]
        for i in range(d):
            for j in range(i+1, d):
                uX = X[:, i] * X[:, j]
                uY = Y[:, i] * Y[:, j]
                K_pair *= np.cos(gamma_pair * (uX[:, None] - uY[None, :]))

        return K_single * K_pair

    return zz_kernel

# ============================================================
# A) Baseline multiclass SVM
# ============================================================
def run_multiclass_svm(
    encoding="cartesian",
    scaler="robust",
    kernel="rbf",             # "rbf" or "zz"
    gamma_single=0.05,
    gamma_pair=0.05,
    C=1.0,
    test_size=0.25,
    seed=42
):
    Xc, y_bits = load_dataset()
    X = make_features(Xc, encoding=encoding)
    y = bits_to_int_labels(y_bits)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    sc = make_scaler(scaler)
    if kernel == "rbf":
        clf = SVC(kernel="rbf", C=C, gamma="scale", decision_function_shape="ovr")
    elif kernel == "zz":
        clf = SVC(kernel=make_zz_kernel(gamma_single, gamma_pair),
                  C=C, decision_function_shape="ovr")
    else:
        raise ValueError("kernel must be 'rbf' or 'zz'")

    if sc is None:
        model = clf
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    else:
        model = Pipeline([("scaler", sc), ("clf", clf)])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return acc, cm

# ============================================================
# B) Hierarchical decoding (4 binary + meta-corrector)
# ============================================================
def run_hierarchical_decoding(
    encoding="cartesian",
    scaler="robust",
    base_model="svm_rbf",   # "svm_rbf" or "mlp"
    meta_corrector=True,
    test_size=0.25,
    seed=42
):
    """
    Step 1: train 4 binary classifiers: predict q1..q4 from full feature vector.
    Step 2: combine predicted bits -> 16-class label.
    Step 3 (optional): meta-corrector trained on predicted probabilities to correct errors.
    """

    Xc, y_bits = load_dataset()
    X = make_features(Xc, encoding=encoding)
    y16 = bits_to_int_labels(y_bits)

    X_train, X_test, y_train_bits, y_test_bits, y_train16, y_test16 = train_test_split(
        X, y_bits, y16, test_size=test_size, random_state=seed, stratify=y16
    )

    sc = make_scaler(scaler)
    if sc is not None:
        sc.fit(X_train)
        Xtr = sc.transform(X_train)
        Xte = sc.transform(X_test)
    else:
        Xtr, Xte = X_train, X_test

    # --- Train 4 binary models ---
    bin_models = []
    bin_proba_tr = []
    bin_proba_te = []

    for qi in range(4):
        ytr = y_train_bits[:, qi]
        yte = y_test_bits[:, qi]

        if base_model == "svm_rbf":
            m = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True)
        elif base_model == "mlp":
            m = MLPClassifier(hidden_layer_sizes=(64, 32),
                              activation="relu",
                              max_iter=300,
                              random_state=seed)
        else:
            raise ValueError("base_model must be 'svm_rbf' or 'mlp'")

        m.fit(Xtr, ytr)

        # predicted probabilities for class=1
        ptr = m.predict_proba(Xtr)[:, 1]
        pte = m.predict_proba(Xte)[:, 1]

        bin_models.append(m)
        bin_proba_tr.append(ptr.reshape(-1, 1))
        bin_proba_te.append(pte.reshape(-1, 1))

    P_tr = np.hstack(bin_proba_tr)  # (Ntrain,4)
    P_te = np.hstack(bin_proba_te)  # (Ntest,4)

    # Hard bit decisions
    pred_bits = (P_te >= 0.5).astype(int)
    pred16 = bits_to_int_labels(pred_bits)

    acc_base = accuracy_score(y_test16, pred16)

    if not meta_corrector:
        return acc_base, None

    # --- Meta-corrector: learn mapping from probabilities to 16-class ---
    # Use multinomial logistic regression on 4 probability features (or add more later)
    meta = LogisticRegression(max_iter=2000, multi_class="multinomial")
    meta.fit(P_tr, y_train16)
    pred16_meta = meta.predict(P_te)
    acc_meta = accuracy_score(y_test16, pred16_meta)

    return acc_base, acc_meta

# ============================================================
# C) Multiclass MLP discriminator
# ============================================================
def run_multiclass_mlp(
    encoding="polar_cross",
    scaler="robust",
    hidden=(128, 64),
    alpha=1e-4,
    max_iter=400,
    test_size=0.25,
    seed=42
):
    Xc, y_bits = load_dataset()
    X = make_features(Xc, encoding=encoding)
    y = bits_to_int_labels(y_bits)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    sc = make_scaler(scaler)

    clf = MLPClassifier(hidden_layer_sizes=hidden,
                        activation="relu",
                        alpha=alpha,
                        max_iter=max_iter,
                        random_state=seed)

    if sc is None:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
    else:
        model = Pipeline([("scaler", sc), ("clf", clf)])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return acc, cm

# ============================================================
# Convenience runner
# ============================================================
def main():
    # 1) Baseline SVM (ZZ) - your current best idea
    acc_zz, _ = run_multiclass_svm(
        encoding="cartesian",
        scaler="robust",
        kernel="zz",
        gamma_single=0.05,
        gamma_pair=0.05,
        C=1.0
    )
    print(f"[A] Multiclass SVM (ZZ kernel) acc = {acc_zz:.4f}")

    # 2) Hierarchical decoding
    acc_bits, acc_meta = run_hierarchical_decoding(
        encoding="cartesian",
        scaler="robust",
        base_model="svm_rbf",
        meta_corrector=True
    )
    print(f"[B] Hierarchical: combine 4 binaries acc = {acc_bits:.4f}")
    print(f"[B] Hierarchical + meta-corrector acc = {acc_meta:.4f}")

    # 3) Multiclass MLP with richer features
    acc_mlp, _ = run_multiclass_mlp(
        encoding="polar_cross",
        scaler="robust",
        hidden=(128, 64),
        alpha=1e-4,
        max_iter=500
    )
    print(f"[C] Multiclass MLP (polar_cross) acc = {acc_mlp:.4f}")


if __name__ == "__main__":
    main()


[A] Multiclass SVM (ZZ kernel) acc = 0.3755


c:\Users\10\anaconda3\envs\qcomputing\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[B] Hierarchical: combine 4 binaries acc = 0.3872
[B] Hierarchical + meta-corrector acc = 0.3862
[C] Multiclass MLP (polar_cross) acc = 0.2848


c:\Users\10\anaconda3\envs\qcomputing\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [2]:
"""
Contextual Bandit (Neural / Linear UCB style) for 4-qubit (16-class) readout.

Offline setting:
  - context x : feature vector extracted from IQ readout
  - action a  : predicted 4-qubit state (0..15)
  - reward r  : 1 if correct, 0 otherwise

This implementation uses Linear UCB per class (action).
Fully compatible with fixed dataset_X.txt / dataset_y.txt.

Why this works:
  - uncertainty-aware decision making
  - robust to class overlap
  - RL-inspired without requiring environment dynamics
"""

import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix


# -------------------------------------------------
# Data loading and labels
# -------------------------------------------------
def load_dataset(X_path="dataset_X.txt", y_path="dataset_y.txt"):
    X = np.loadtxt(X_path, dtype=complex)  # (N,4)
    y_bits = np.loadtxt(y_path, dtype=int) # (N,4)
    return X, y_bits


def bits_to_int_labels(y_bits):
    q1, q2, q3, q4 = (y_bits[:, i] for i in range(4))
    return (q1 + 2*q2 + 4*q3 + 8*q4).astype(int)


# -------------------------------------------------
# Feature engineering
# -------------------------------------------------
def features_cartesian(Xc):
    return np.hstack([Xc.real, Xc.imag])  # (N,8)


def features_polar(Xc):
    amp = np.abs(Xc)
    phi = np.angle(Xc)
    return np.hstack([amp, np.sin(phi), np.cos(phi)])  # (N,12)


def features_cross(Xc):
    feats = []
    for i, j in combinations(range(4), 2):
        prod = Xc[:, i] * np.conjugate(Xc[:, j])
        feats.append(prod.real.reshape(-1, 1))
        feats.append(prod.imag.reshape(-1, 1))
    return np.hstack(feats)  # (N,12)


def make_features(Xc, mode="polar_cross"):
    if mode == "cartesian":
        return features_cartesian(Xc)
    if mode == "polar":
        return features_polar(Xc)
    if mode == "polar_cross":
        return np.hstack([features_polar(Xc), features_cross(Xc)])
    raise ValueError("Invalid feature mode")


# -------------------------------------------------
# Linear UCB Bandit
# -------------------------------------------------
class LinearUCBBandit:
    def __init__(self, n_actions, n_features, alpha=1.0):
        self.n_actions = n_actions
        self.n_features = n_features
        self.alpha = alpha

        self.A = [np.eye(n_features) for _ in range(n_actions)]
        self.b = [np.zeros(n_features) for _ in range(n_actions)]

    def predict(self, x):
        """
        Select action using UCB rule.
        """
        scores = np.zeros(self.n_actions)

        for a in range(self.n_actions):
            Ainv = np.linalg.inv(self.A[a])
            theta = Ainv @ self.b[a]
            mean = theta @ x
            uncertainty = self.alpha * np.sqrt(x @ Ainv @ x)
            scores[a] = mean + uncertainty

        return np.argmax(scores)

    def update(self, action, x, reward):
        self.A[action] += np.outer(x, x)
        self.b[action] += reward * x


# -------------------------------------------------
# Training & evaluation
# -------------------------------------------------
def run_contextual_bandit(
    feature_mode="polar_cross",
    alpha=1.0,
    test_size=0.25,
    seed=42
):
    # Load data
    Xc, y_bits = load_dataset()
    y = bits_to_int_labels(y_bits)

    X = make_features(Xc, mode=feature_mode)

    # Scale features (important!)
    scaler = RobustScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    n_features = X.shape[1]
    n_actions = 16

    bandit = LinearUCBBandit(
        n_actions=n_actions,
        n_features=n_features,
        alpha=alpha
    )

    # --- Offline training ---
    for x, y_true in zip(X_train, y_train):
        action = bandit.predict(x)
        reward = 1.0 if action == y_true else 0.0
        bandit.update(action, x, reward)

    # --- Evaluation ---
    y_pred = np.array([bandit.predict(x) for x in X_test])

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n=== Contextual Bandit (Linear UCB) ===")
    print(f"Feature mode : {feature_mode}")
    print(f"Alpha        : {alpha}")
    print(f"Accuracy     : {acc:.4f}")

    return acc, cm


# -------------------------------------------------
# Main
# -------------------------------------------------
if __name__ == "__main__":
    # Try different alphas (exploration strength)
    for alpha in [0.1, 0.3, 0.5, 1.0]:
        run_contextual_bandit(
            feature_mode="polar_cross",
            alpha=alpha
        )



=== Contextual Bandit (Linear UCB) ===
Feature mode : polar_cross
Alpha        : 0.1
Accuracy     : 0.3663

=== Contextual Bandit (Linear UCB) ===
Feature mode : polar_cross
Alpha        : 0.3
Accuracy     : 0.3595

=== Contextual Bandit (Linear UCB) ===
Feature mode : polar_cross
Alpha        : 0.5
Accuracy     : 0.3688

=== Contextual Bandit (Linear UCB) ===
Feature mode : polar_cross
Alpha        : 1.0
Accuracy     : 0.3510


In [ ]:
"""
Experimental model for 4-qubit (16-class) readout classification using:
  1) Angle (polar) encoding with sin/cos phase features
  2) Cross-qubit interaction features Re(z_i * conj(z_j)), Im(z_i * conj(z_j))
  3) A small MLP (FNN discriminator) with regularization

Data:
  - dataset_X.txt: (N,4) complex readouts
  - dataset_y.txt: (N,4) bits [q1,q2,q3,q4], where q1 is the rightmost bit (LSB)

This script:
  - builds feature matrix X_feat
  - scales it using RobustScaler
  - trains an MLPClassifier for 16-class prediction
  - reports accuracy + confusion matrix
"""
import numpy as np
from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# -------------------------------------------------
# Load dataset
# -------------------------------------------------
def load_dataset(X_path="dataset_X.txt", y_path="dataset_y.txt"):
    X = np.loadtxt(X_path, dtype=complex)   # (N,4)
    y_bits = np.loadtxt(y_path, dtype=int)  # (N,4) with columns [q1,q2,q3,q4]
    return X, y_bits


def bits_to_int_labels(y_bits):
    """
    Convert [q1,q2,q3,q4] -> integer 0..15, with q1 as LSB and q4 as MSB.
    """
    q1, q2, q3, q4 = (y_bits[:, i] for i in range(4))
    return (q1 + 2*q2 + 4*q3 + 8*q4).astype(int)


# -------------------------------------------------
# Feature engineering: Polar + Cross
# -------------------------------------------------
def polar_features(Xc):
    """
    For each qubit z = I + iQ:
      amp = |z|
      phi = angle(z)
    Use sin(phi), cos(phi) to avoid phase discontinuity.
    Output dims: 4*(1+1+1) = 12
      [amp1..amp4, sin(phi1)..sin(phi4), cos(phi1)..cos(phi4)]
    """
    amp = np.abs(Xc)
    phi = np.angle(Xc)
    return np.hstack([amp, np.sin(phi), np.cos(phi)])


def cross_features(Xc):
    """
    Cross-qubit interaction features using z_i * conj(z_j).
    For 4 qubits there are C(4,2)=6 pairs, each gives Re and Im => 12 dims.
    """
    feats = []
    for i, j in combinations(range(4), 2):
        prod = Xc[:, i] * np.conjugate(Xc[:, j])
        feats.append(prod.real.reshape(-1, 1))
        feats.append(prod.imag.reshape(-1, 1))
    return np.hstack(feats)  # (N,12)


def build_features(Xc, mode="polar_cross"):
    if mode == "polar":
        return polar_features(Xc)                 # (N,12)
    if mode == "polar_cross":
        return np.hstack([polar_features(Xc), cross_features(Xc)])  # (N,24)
    if mode == "cartesian":
        return np.hstack([Xc.real, Xc.imag])      # (N,8)
    raise ValueError("mode must be 'cartesian'|'polar'|'polar_cross'")


# -------------------------------------------------
# Train & evaluate MLP for 16-class prediction
# -------------------------------------------------
def train_mlp_multiclass(
    feature_mode="polar_cross",
    hidden_layers=(128, 64),
    alpha=1e-4,
    max_iter=600,
    test_size=0.25,
    seed=42
):
    # Load
    Xc, y_bits = load_dataset()
    y = bits_to_int_labels(y_bits)

    # Features
    X = build_features(Xc, mode=feature_mode)

    # Scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    # MLP classifier
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layers,
        activation="relu",
        solver="adam",
        alpha=alpha,          # L2 regularization strength
        max_iter=max_iter,
        random_state=seed,
        early_stopping=True,
        n_iter_no_change=25,
        validation_fraction=0.15
    )

    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n=== MLP Multiclass (16-state) ===")
    print(f"Feature mode : {feature_mode}")
    print(f"Hidden layers: {hidden_layers}")
    print(f"alpha (L2)   : {alpha}")
    print(f"Accuracy     : {acc:.4f}")

    # Optional: more detailed report
    print("\nClassification report (macro avg is informative for balanced classes):")
    print(classification_report(y_test, y_pred, digits=3))

    return mlp, scaler, acc, cm


# -------------------------------------------------
# Quick benchmark: compare feature modes
# -------------------------------------------------
def benchmark_feature_modes():
    for mode in ["cartesian", "polar", "polar_cross"]:
        _, _, acc, _ = train_mlp_multiclass(
            feature_mode=mode,
            hidden_layers=(128, 64),
            alpha=1e-4,
            max_iter=700
        )
        print(f"[Benchmark] mode={mode:12s}  acc={acc:.4f}")


# -------------------------------------------------
# Run
# -------------------------------------------------
if __name__ == "__main__":
    # 1) Train one strong candidate:
    train_mlp_multiclass(
        feature_mode="polar_cross",
        hidden_layers=(128, 64),
        alpha=1e-4,
        max_iter=700
    )

    # 2) Optional: compare all 3 feature modes quickly:
    # benchmark_feature_modes()



=== MLP Multiclass (16-state) ===
Feature mode : polar_cross
Hidden layers: (128, 64)
alpha (L2)   : 0.0001
Accuracy     : 0.3810

Classification report (macro avg is informative for balanced classes):
              precision    recall  f1-score   support

           0      0.316     0.648     0.425       250
           1      0.325     0.344     0.334       250
           2      0.341     0.440     0.384       250
           3      0.420     0.368     0.392       250
           4      0.331     0.448     0.381       250
           5      0.505     0.404     0.449       250
           6      0.396     0.380     0.388       250
           7      0.431     0.248     0.315       250
           8      0.424     0.500     0.459       250
           9      0.420     0.284     0.339       250
          10      0.379     0.376     0.378       250
          11      0.437     0.348     0.388       250
          12      0.438     0.420     0.429       250
          13      0.327     0.324     0.

In [ ]:
"""
NeuralUCB for 4-qubit (16-class) readout classification
Offline contextual bandit setting.

- context x : feature vector from IQ readout
- action a  : class in {0..15}
- reward r  : 1 if correct, 0 otherwise

Implements NeuralUCB with:
  - shared neural network
  - action encoded as one-hot
  - Jacobian-based uncertainty estimate
"""

import numpy as np
from itertools import combinations

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix
# -------------------------------------------------
# Data & features
# -------------------------------------------------
def load_dataset(X_path="dataset_X.txt", y_path="dataset_y.txt"):
    X = np.loadtxt(X_path, dtype=complex)
    y_bits = np.loadtxt(y_path, dtype=int)
    return X, y_bits

def bits_to_int_labels(y_bits):
    q1, q2, q3, q4 = (y_bits[:, i] for i in range(4))
    return (q1 + 2*q2 + 4*q3 + 8*q4).astype(int)

def features_polar(Xc):
    amp = np.abs(Xc)
    phi = np.angle(Xc)
    return np.hstack([amp, np.sin(phi), np.cos(phi)])  # (N,12)

def features_cross(Xc):
    feats = []
    for i, j in combinations(range(4), 2):
        prod = Xc[:, i] * np.conjugate(Xc[:, j])
        feats.append(prod.real.reshape(-1, 1))
        feats.append(prod.imag.reshape(-1, 1))
    return np.hstack(feats)  # (N,12)

def make_features(Xc, mode="polar_cross"):
    if mode == "polar_cross":
        return np.hstack([features_polar(Xc), features_cross(Xc)])
    raise ValueError("Only polar_cross used for NeuralUCB")

# -------------------------------------------------
# Neural reward model
# -------------------------------------------------
class RewardNet(nn.Module):
    def __init__(self, input_dim, hidden=(128, 64)):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden[0]),
            nn.ReLU(),
            nn.Linear(hidden[0], hidden[1]),
            nn.ReLU(),
            nn.Linear(hidden[1], 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)

# -------------------------------------------------
# NeuralUCB Agent
# -------------------------------------------------
class NeuralUCB:
    def __init__(self, context_dim, n_actions=16, alpha=1.0, lr=1e-3):
        self.n_actions = n_actions
        self.alpha = alpha

        self.model = RewardNet(context_dim + n_actions)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()

        # A = λI initially
        self.params = list(self.model.parameters())
        self.p_dim = sum(p.numel() for p in self.params)
        self.A = torch.eye(self.p_dim)

    def _one_hot(self, a):
        v = torch.zeros(self.n_actions)
        v[a] = 1.0
        return v

    def _forward(self, x, a):
        xa = torch.cat([x, self._one_hot(a)])
        return self.model(xa)

    def _grad_vector(self, x, a):
        self.optimizer.zero_grad()
        r = self._forward(x, a)
        r.backward()

        grads = []
        for p in self.model.parameters():
            grads.append(p.grad.view(-1))
        return torch.cat(grads).detach()

    def select_action(self, x):
        scores = []

        Ainv = torch.inverse(self.A)

        for a in range(self.n_actions):
            with torch.no_grad():
                mean = self._forward(x, a)

            g = self._grad_vector(x, a)
            uncert = torch.sqrt(g @ Ainv @ g)
            score = mean + self.alpha * uncert
            scores.append(score.item())

        return int(np.argmax(scores))

    def update(self, x, a, reward):
        # Gradient for uncertainty
        g = self._grad_vector(x, a)
        self.A += torch.outer(g, g)

        # Train reward model
        self.optimizer.zero_grad()
        pred = self._forward(x, a)
        loss = self.loss_fn(pred, torch.tensor(reward))
        loss.backward()
        self.optimizer.step()

# -------------------------------------------------
# Training & evaluation
# -------------------------------------------------
def run_neural_ucb(
    alpha=0.5,
    test_size=0.25,
    seed=42
):
    torch.manual_seed(seed)
    np.random.seed(seed)

    Xc, y_bits = load_dataset()
    y = bits_to_int_labels(y_bits)

    X = make_features(Xc, mode="polar_cross")

    scaler = RobustScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    Xtr = torch.tensor(X_train, dtype=torch.float32)
    Xte = torch.tensor(X_test, dtype=torch.float32)

    agent = NeuralUCB(
        context_dim=Xtr.shape[1],
        n_actions=16,
        alpha=alpha
    )

    # --- Offline training ---
    for x, y_true in zip(Xtr, y_train):
        a = agent.select_action(x)
        r = 1.0 if a == y_true else 0.0
        agent.update(x, a, r)

    # --- Evaluation ---
    preds = []
    for x in Xte:
        preds.append(agent.select_action(x))

    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    print("\n=== NeuralUCB (offline) ===")
    print(f"Alpha    : {alpha}")
    print(f"Accuracy : {acc:.4f}")

    return acc, cm

# -------------------------------------------------
# Main
# -------------------------------------------------
# if __name__ == "__main__":
#     for alpha in [0.1, 0.3, 0.5, 1.0]:
#         run_neural_ucb(alpha=alpha)

In [14]:
import os

def save_checkpoint(agent, step, filename="neuralucb_ckpt.pt"):
    torch.save({
        "model_state": agent.model.state_dict(),
        "A": agent.A,
        "step": step
    }, filename)
    print(f"[Checkpoint saved at step {step}]")

def load_checkpoint(agent, filename="neuralucb_ckpt.pt"):
    if not os.path.exists(filename):
        print("[No checkpoint found — starting fresh]")
        return 0

    ckpt = torch.load(filename, map_location="cpu")
    agent.model.load_state_dict(ckpt["model_state"])
    agent.A = ckpt["A"]
    step = ckpt["step"]

    print(f"[Checkpoint loaded — resuming from step {step}]")
    return step


In [17]:
def run_neural_ucb(
    alpha=0.5,
    test_size=0.25,
    seed=42,
    max_steps_per_run=50,     # ⬅️ چند شات در هر اجرا
    checkpoint_file="neuralucb_ckpt.pt"
):
    torch.manual_seed(seed)
    np.random.seed(seed)

    Xc, y_bits = load_dataset()
    y = bits_to_int_labels(y_bits)

    X = make_features(Xc, mode="polar_cross")

    scaler = RobustScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=seed, stratify=y
    )

    Xtr = torch.tensor(X_train, dtype=torch.float32)
    Xte = torch.tensor(X_test, dtype=torch.float32)

    agent = NeuralUCB(
        context_dim=Xtr.shape[1],
        n_actions=16,
        alpha=alpha
    )

    # ⬇️ تلاش برای ادامه از checkpoint
    start_step = load_checkpoint(agent, checkpoint_file)

    # --------- Offline training (مرحله‌ای) ---------
    end_step = min(start_step + max_steps_per_run, len(Xtr))

    for i in range(start_step, end_step):
        x = Xtr[i]
        y_true = y_train[i]

        a = agent.select_action(x)
        r = 1.0 if a == y_true else 0.0
        agent.update(x, a, r)

        if i % 10 == 0:
            print(f"Training step {i}/{len(Xtr)}")

    # ⬇️ ذخیره‌ی امن
    save_checkpoint(agent, end_step, checkpoint_file)

    print(f"[Stopped safely at step {end_step}]")

    # --------- Evaluation فقط وقتی آموزش کامل شد ---------
    if end_step < len(Xtr):
        print("Training not finished yet — skipping evaluation")
        return None, None

    preds = []
    for x in Xte:
        preds.append(agent.select_action(x))

    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)

    print("\n=== NeuralUCB (offline, finished) ===")
    print(f"Alpha    : {alpha}")
    print(f"Accuracy : {acc:.4f}")

    return acc, cm


In [19]:
import torch
import numpy as np

run_neural_ucb(
    alpha=0.5,
    test_size=0.25,
    seed=42,
    max_steps_per_run=50,     # ⬅️ چند شات در هر اجرا
    checkpoint_file="neuralucb_ckpt.pt"
)

[Checkpoint loaded — resuming from step 50]
Training step 50/12000
Training step 60/12000
Training step 70/12000
Training step 80/12000
Training step 90/12000
[Checkpoint saved at step 100]
[Stopped safely at step 100]
Training not finished yet — skipping evaluation


(None, None)